In [1]:
# Required Libraries
import re
import unicodedata
import nltk
import string
from nltk.corpus import stopwords
from collections import Counter
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding, SpatialDropout1D
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix, accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
from keras.utils import to_categorical

# # Download NLTK stopwords
#nltk.download('stopwords')
#nltk.download('punkt')

ModuleNotFoundError: No module named 'tensorflow'

## 2. Use the data.csv dataset as provided as pandas DataFrame and  process the text feature as questioned below.

In [9]:
df = pd.read_csv('data_set.csv')
print("Dataset dimension is : ", df.shape)
print("Dataset Smaple ", )
df.head()

Dataset dimension is :  (179, 4)
Dataset Smaple 


,Unnamed: 0,Id,Text,Score
0,0,3306,"I HATE SUCRALOSE.<br /><br />Honestly, I don't...",3
1,1,7851,I love rice and this was good with all kinds o...,5
2,2,17227,I would like to tell this company that the who...,1
3,3,18152,The label says that this product is made by re...,2
4,4,21017,I ordered the GREENIES 'PETITE' size for my do...,3


2 a) Pre-Process the text feature as questioned below. (in the same sequence)


2 a) 1.	Remove the accented characters from text feature. (3 marks)


In [10]:
def remove_accented_char(text):
    return unicodedata.normalize('NFKD',text).encode('ascii',errors='ignore').decode('utf-8',errors='ignore')

text='I am Satish. Pläne für die Freizeit'
remove_accented_char(text)

'I am Satish. Plane fur die Freizeit'

2) a) 2)	Remove digits from text feature. (3)

In [11]:
def remove_digit(text):
    return re.sub(r'\d+|.\d+', '', text)

print(remove_digit('I am 22.6. You are 23. We are .36. '))

I am. You are. We are . 


2) a) 3)	Remove punctuations from text feature. (3)

In [31]:
# 5. Remove punctuations
def remove_punctuations(text):
    text = re.sub(r'[^\w\s]', '', text)
    return text

remove_punctuations('You\'re are printing')

'Youre are printing'

In [15]:
import contractions

def remove_punctuations(text):
    text = re.sub(r'[^\w\s]', '', text)

remove_contractions("You're invited. I'm watching. We'll do it.")

'You are invited. I am watching. We will do it.'

2.a) 4)	Remove stopwords from text feature.  (3)

In [16]:
def remove_stopwords(text):
    swords = stopwords.words('english')
    tokens = text.split()
    tokens = [x for x in tokens if x.lower() not in swords]
    return " ".join(tokens)

remove_stopwords("We are working towards the solution through the project Xenab")

'working towards solution project Xenab'

2) a) 5)	Eliminate multiple spaces from text feature. (3)

In [27]:
def remove_multiple_space(text):
    return re.sub(r"\s+", " ", text).strip()

remove_multiple_space("  I    am   Satishkumar venk.   My work is index. I")

'I am Satishkumar venk. My work is index. I'

2) b) Find out the 5 most frequent words in the text corpus ((from the preprocessed output of previous question ) (8 Marks)

In [32]:
def preprocess_text(text):
    text = remove_accented_char(text)
    text = remove_digit(text)
    text = remove_contractions(text)
    text = remove_punctuations(text)
    text = remove_stopwords(text)
    text = remove_multiple_space(text)

    return text

In [35]:
df['Cleaned_Text'] = df['Text'].apply(preprocess_text)
df['Cleaned_Text']

0      HATE SUCRALOSEbr br Honestly know people CANNO...
1      love rice good kinds textures allergic many th...
2      would like tell company whole box bars melted ...
3      label says product made removing water FAT pea...
4      ordered GREENIES PETITE size dogs done many ti...
                             ...                        
174    bread little sweet crumbled followed direction...
175    Chocolate tastes great goes without sayingI me...
176    got bag brown squished Jelly Bellies Maybe bag...
177    wanted omegas daughter would eat really smells...
178    looks smells great son really like texture muc...
Name: Cleaned_Text, Length: 179, dtype: object

2) c) Vectorize the pre-processed text feature by building/training a Skip-Gram Word2Vec model. Use this Skip-Gram Word2Vec model to fetch the top 5 most similar word for the word 'food'.  (marks 3+5 =8)

In [34]:
from gensim.models import Word2Vec

In [37]:
sentences = [sentence.split() for sentence in df['Cleaned_Text']]

In [40]:
# sg=1 causes Word2Vec to use Skip-Gram model
wvec_skipgram = Word2Vec(sentences=sentences, vector_size=100, min_count=5, sg=1)
wvec_skipgram.wv.most_similar('food',topn=5)

[('like', 0.9983640313148499),
 ('br', 0.9980814456939697),
 ('flavor', 0.9980767369270325),
 ('product', 0.9979872703552246),
 ('taste', 0.9979698061943054)]

2) d) Vectorize the pre-processed text feature by building a CBOW Word2Vec model. Use the trained CBOW Word2Vec model to fetch the top 5 most similar word for the word 'food'. Is the output different than Skip-Gram’s output? ( marks 5+3+1)

In [41]:
# sg=0 causes Word2Vec to use CBOW (Continuous Bag of Words) model
wvec_cbow = Word2Vec(sentences=sentences, vector_size=100, min_count=5, sg=0)
wvec_cbow.wv.most_similar('food',topn=5)

[('like', 0.7372711896896362),
 ('flavor', 0.6920003890991211),
 ('oatmeal', 0.689063549041748),
 ('product', 0.6852536797523499),
 ('br', 0.6769256591796875)]

## 3. Model Building

3) a)  Convert Textual output ( of question 2. a) into numerical using countvectorizer  (marks 8)

In [42]:
model_cntvector = CountVectorizer()
cntvectors = model_cntvector.fit_transform(df['Cleaned_Text'])
cntvectors.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

3) b) Convert Textual output ( of question 2. a) into numerical using  TfidfVectorizer (marks 8)

In [43]:
model_tfidf = TfidfVectorizer()
tfidf_vectors = model_tfidf.fit_transform(df['Cleaned_Text'])
tfidf_vectors.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

3) c) Build LSTM multiclass text classification model on the cleaned dataset (output of question 2. a) using Keras libraries.  (marks 20)


In [89]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, SpatialDropout1D

In [90]:
df

,Unnamed: 0,Id,Text,Score,Cleaned_Text
0,0,3306,"I HATE SUCRALOSE.<br /><br />Honestly, I don't...",3,HATE SUCRALOSEbr br Honestly know people CANNO...
1,1,7851,I love rice and this was good with all kinds o...,5,love rice good kinds textures allergic many th...
2,2,17227,I would like to tell this company that the who...,1,would like tell company whole box bars melted ...
3,3,18152,The label says that this product is made by re...,2,label says product made removing water FAT pea...
4,4,21017,I ordered the GREENIES 'PETITE' size for my do...,3,ordered GREENIES PETITE size dogs done many ti...
...,...,...,...,...,...
174,174,548557,The bread was a little sweet and it crumbled. ...,3,bread little sweet crumbled followed direction...
175,175,557657,Chocolate tastes great! But that goes without ...,4,Chocolate tastes great goes without sayingI me...
176,176,559007,"We got a bag of brown, squished Jelly Bellies....",1,got bag brown squished Jelly Bellies Maybe bag...
177,177,567957,I wanted this for the omega 3's but my daughte...,2,wanted omegas daughter would eat really smells...


In [91]:
X = df['Cleaned_Text']
y = pd.get_dummies(df['Score'], dtype='int32')
y

,1,2,3,4,5
0,0,0,1,0,0
1,0,0,0,0,1
2,1,0,0,0,0
3,0,1,0,0,0
4,0,0,1,0,0
...,...,...,...,...,...
174,0,0,1,0,0
175,0,0,0,1,0
176,1,0,0,0,0
177,0,1,0,0,0


In [92]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding, SpatialDropout1D
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import LSTM

In [93]:
max_words = 5000
max_len = 100

tokenizer = Tokenizer(num_words=max_words, lower=True)
tokenizer.fit_on_texts(X)
X_tokenized = tokenizer.texts_to_sequences(X)
X_padded = pad_sequences(X_tokenized, maxlen=max_len)
X_padded.shape

X_train, X_test, y_train, y_test = train_test_split(X_padded, y, test_size=0.3, random_state=43)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((125, 100), (54, 100), (125, 5), (54, 5))

In [94]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [95]:
model.fit(X_train, y_train, batch_size=65, epochs=10, validation_data=[X_test, y_test], verbose=2)

Epoch 1/10
2/2 - 1s - 367ms/step - accuracy: 0.9360 - loss: 0.4588 - val_accuracy: 0.1481 - val_loss: 1.7724
Epoch 2/10
2/2 - 0s - 57ms/step - accuracy: 0.9440 - loss: 0.4354 - val_accuracy: 0.1667 - val_loss: 1.7275
Epoch 3/10
2/2 - 0s - 59ms/step - accuracy: 0.9360 - loss: 0.3459 - val_accuracy: 0.2407 - val_loss: 1.6505
Epoch 4/10
2/2 - 0s - 58ms/step - accuracy: 0.9760 - loss: 0.3091 - val_accuracy: 0.2222 - val_loss: 1.6614
Epoch 5/10
2/2 - 0s - 57ms/step - accuracy: 0.9520 - loss: 0.2869 - val_accuracy: 0.2593 - val_loss: 1.6888
Epoch 6/10
2/2 - 0s - 56ms/step - accuracy: 0.9760 - loss: 0.2592 - val_accuracy: 0.2222 - val_loss: 1.7031
Epoch 7/10
2/2 - 0s - 58ms/step - accuracy: 0.9760 - loss: 0.2488 - val_accuracy: 0.2593 - val_loss: 1.6895
Epoch 8/10
2/2 - 0s - 60ms/step - accuracy: 1.0000 - loss: 0.1805 - val_accuracy: 0.2963 - val_loss: 1.6882
Epoch 9/10
2/2 - 0s - 64ms/step - accuracy: 1.0000 - loss: 0.1587 - val_accuracy: 0.3148 - val_loss: 1.7049
Epoch 10/10
2/2 - 0s - 62ms

In [96]:
y_pred = model.predict(X_test)
y_pred_result = np.argmax(y_pred, axis=1)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step


In [97]:
y_pred_result

array([4, 2, 4, 4, 3, 4, 1, 2, 3, 3, 0, 4, 1, 1, 2, 2, 2, 4, 3, 3, 0, 3,
       4, 3, 3, 3, 3, 0, 4, 4, 2, 2, 4, 3, 1, 2, 3, 3, 0, 0, 4, 1, 3, 2,
       0, 0, 3, 4, 2, 0, 2, 2, 3, 3])

In [98]:
y_test_result = np.argmax(y_test, axis=1)
y_test_result

array([1, 0, 0, 4, 2, 4, 2, 0, 4, 2, 3, 0, 0, 1, 2, 2, 1, 2, 1, 3, 0, 4,
       4, 3, 4, 4, 1, 0, 0, 4, 0, 0, 3, 1, 4, 2, 4, 2, 4, 0, 4, 4, 2, 3,
       0, 3, 0, 0, 1, 1, 3, 2, 4, 1])

In [99]:
accuracy_score(y_test_result, y_pred_result)

0.2962962962962963

In [100]:
from sklearn.metrics import classification_report

In [101]:
rpt = classification_report(y_pred_result, y_test_result)
print(rpt)

              precision    recall  f1-score   support

           0       0.29      0.50      0.36         8
           1       0.11      0.20      0.14         5
           2       0.40      0.33      0.36        12
           3       0.29      0.12      0.17        17
           4       0.36      0.42      0.38        12

    accuracy                           0.30        54
   macro avg       0.29      0.31      0.28        54
weighted avg       0.31      0.30      0.29        54



3) D) show the confusion matrix and compute accuracy from the model and interpret it.  (marks 4)

In [ ]:
# Build the LSTM model
model = Sequential()
model.add(Embedding(max_words, 128))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(5, activation='softmax'))  # 3 classes for sentiment

In [ ]:
history = model.fit(X_train, y_train, epochs=5, batch_size=64, validation_data=(X_test, y_test), verbose=2)

In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, SpatialDropout1D

In [50]:
from tensorflow.keras.layers import Bidirectional

# BILSTM model

In [105]:
bilstm_model = Sequential()
bilstm_model.add(Embedding(max_words, 128))
bilstm_model.add(SpatialDropout1D(0.2))
bilstm_model.add(Bidirectional(LSTM(10, return_sequences=True)))
bilstm_model.add(Bidirectional(LSTM(10)))
bilstm_model.add(Dense(5, activation="softmax"))

bilstm_model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

In [106]:
bilstm_model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spatial_dropout1d_3             │ ?                      │   0 (unbuilt) │
│ (SpatialDropout1D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_2 (Bidirectional) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_3 (Bidirectional) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [107]:
bilstm_model.fit(X_train, y_train, batch_size=60, epochs=10, validation_data=[X_test, y_test], verbose=2)

Epoch 1/10
3/3 - 2s - 685ms/step - accuracy: 0.1760 - loss: 1.6116 - val_accuracy: 0.1852 - val_loss: 1.6130
Epoch 2/10
3/3 - 0s - 26ms/step - accuracy: 0.3440 - loss: 1.5972 - val_accuracy: 0.1296 - val_loss: 1.6103
Epoch 3/10
3/3 - 0s - 26ms/step - accuracy: 0.3760 - loss: 1.5880 - val_accuracy: 0.2593 - val_loss: 1.6076
Epoch 4/10
3/3 - 0s - 25ms/step - accuracy: 0.3360 - loss: 1.5770 - val_accuracy: 0.2593 - val_loss: 1.6067
Epoch 5/10
3/3 - 0s - 26ms/step - accuracy: 0.3600 - loss: 1.5644 - val_accuracy: 0.2593 - val_loss: 1.6073
Epoch 6/10
3/3 - 0s - 25ms/step - accuracy: 0.4000 - loss: 1.5524 - val_accuracy: 0.2593 - val_loss: 1.6080
Epoch 7/10
3/3 - 0s - 26ms/step - accuracy: 0.4800 - loss: 1.5304 - val_accuracy: 0.2593 - val_loss: 1.6105
Epoch 8/10
3/3 - 0s - 26ms/step - accuracy: 0.7120 - loss: 1.5086 - val_accuracy: 0.1481 - val_loss: 1.6146
Epoch 9/10
3/3 - 0s - 26ms/step - accuracy: 0.7920 - loss: 1.4818 - val_accuracy: 0.1296 - val_loss: 1.6198
Epoch 10/10
3/3 - 0s - 25ms

In [110]:
y_pred = model.predict(X_test)
y_pred_result = np.argmax(y_pred, axis=1)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step


In [111]:
y_pred_result

array([4, 2, 4, 4, 3, 4, 1, 2, 3, 3, 0, 4, 1, 1, 2, 2, 2, 4, 3, 3, 0, 3,
       4, 3, 3, 3, 3, 0, 4, 4, 2, 2, 4, 3, 1, 2, 3, 3, 0, 0, 4, 1, 3, 2,
       0, 0, 3, 4, 2, 0, 2, 2, 3, 3])

In [112]:
accuracy_score( y_test_result,y_pred_result)

0.2962962962962963

In [ ]:
model = Sequential([
    Input(shape=(5, 10)),
    Bidirectional(LSTM(10, return_sequences=True),
    Bidirectional(LSTM(10)),
    Dense(5, activation="softmax"),
])
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

# With custom backward layer
forward_layer = LSTM(10, return_sequences=True)
backward_layer = LSTM(10, activation='relu', return_sequences=True,
                      go_backwards=True)
model = Sequential([
    Input(shape=(5, 10)),
    Bidirectional(forward_layer, backward_layer=backward_layer),
    Dense(5, activation="softmax"),
])
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')


In [109]:
Dense??

Init signature: Dense(*args, **kwargs)
Source:        
@keras_export("keras.layers.Dense")
class Dense(Layer):
    """Just your regular densely-connected NN layer.

    `Dense` implements the operation:
    `output = activation(dot(input, kernel) + bias)`
    where `activation` is the element-wise activation function
    passed as the `activation` argument, `kernel` is a weights matrix
    created by the layer, and `bias` is a bias vector created by the layer
    (only applicable if `use_bias` is `True`).

    Note: If the input to the layer has a rank greater than 2, `Dense`
    computes the dot product between the `inputs` and the `kernel` along the
    last axis of the `inputs` and axis 0 of the `kernel` (using `tf.tensordot`).
    For example, if input has dimensions `(batch_size, d0, d1)`, then we create
    a `kernel` with shape `(d1, units)`, and the `kernel` operates along axis 2
    of the `input`, on every sub-tensor of shape `(1, 1, d1)` (there are
    `batch_size * d0` such